# Package Building with `skbase`

### Overview of this notebook

building an sklearn-like package

* use of base classes
* strategy pattern, template pattern
* tags and configs
* retrieval via `all_objects`
* heterogeneous meta-estimators
* testing

In [1]:
from os import sys
sys.path.append("..")

In [2]:
import numpy as np
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Using ``skbase`` to implement an ``sklearn``-like

Recommended recipe:

1. implement a few examples of algorithms
2. use that to come up with a sensible template design
3. write down the user contract as a base class sketch, with full docstrings (incl assumes, guarantees)
4. write down the extension contract as an extension template with full docstrings (incl assumes, guarantees)
5. implement base classes using ``BaseObject``, ``BaseEstimator``, meta classes if applicable
    * public interface, boilerplate methods, base functionality
    * template methods, extension interface
    * tags, configs
6. implement concrete classes, start with reinterfacing examples
    * subject to tests
7. implement test framework for interface, e.g., `check_estimator`

Advanced functionality:

* estimator retrieval via `all_objects`
* systematic testing via test class framework

### Iteration 0 - examples, initial design

at the start, we want to:

* implement examples of the algorithms to inform the design
* use the examples to abstract a public unified interface, extender pattern

IMPORTANT: "3 example rule"

= before abstracting a design, ensure you have at least 3 substantially distinct examples!

e.g., 3 regressors, 3 transformers, 3 pipelines, etc

(in this repository, there is only 1 example per, for ease of reading
- but reader is expected to be familiar with others from `sklearn`!)

Examples in the repo:

* linear regression "naive implementation"
* preprocessing/scaling "naive implementation

*[pydata_sktime/mini_sklearn_v0](https://github.com/sktime/sktime-tutorial-pydata-seattle-2023/blob/main/pydata_sktime/mini_sklearn_v1)*

### Iteration 1 - base class interface, strategy pattern

our 1st iteration - class design based on iteration 0

* in `base.py`, we define the public contract
    * `fit`/`predict` for regressors, `fit`/`transform` for transformers
* `base.py` also has the extender contract
    * overriding `fit`/`predict` directly
    * ensuring input and output checks are in place

*[pydata_sktime/mini_sklearn_v1](https://github.com/sktime/sktime-tutorial-pydata-seattle-2023/blob/main/pydata_sktime/mini_sklearn_v1)*

In [ ]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True, as_frame=True)
y = pd.DataFrame(y)

In [ ]:
from pydata_sktime.mini_sklearn_v1.regression import LinReg
from pydata_sktime.mini_sklearn_v1.transform import Scaler

t = Scaler(strategy="std")
clf = LinReg()

In [ ]:
clf.get_params()

In [ ]:
clf.fit(X, y)

In [ ]:
clf.get_fitted_params()

In [ ]:
clf.predict(X)

In [ ]:
t.get_params()

In [ ]:
t.fit(X)

In [ ]:
t.get_fitted_params()

In [ ]:
t.transform(X)

---
## Summary

* `sktime` is an extensible, sklearn-like framework for learning with time series
* use extension template to build `sktime`-compatible components
* use `sktime` testing integration to ensure compatibility and in CI/CD
* `sktime`-compatible components work out-of-the-box with `sktime` framework machinery

---

### Credits: notebook 3 - advanced extension

notebook creation: fkiraly

skbase: fkiraly, rnkuhns 
sktime style design: fkiraly

---

## Join sktime!

* openly governed - users, developers, early career data scientists
* world-wide contributor and user footprint

**EVERYONE CAN JOIN! EVERYONE CAN BECOME A COMMUNITY LEADER!**

* join our discord (developers and community)!
    * regular **community collaboration sessions** and stand-ups on Fridays
    * next **developer sprint**: 

Opportunities:

* sktime **mentoring programme**: github.com/sktime/mentoring

**sktime developer sprint 2023**